In [1]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')
import torch
import torch.nn as nn
from torch.nn.functional import gumbel_softmax,softmax
from models.TestTensors import t_2_2_3

In [13]:
from torch.nn import CosineSimilarity

" example for cosine similarity along the last dimension "
cos = CosineSimilarity(dim=-1)

a = torch.rand((3,2,3))
b = torch.rand((3,2,3))

c = a[0]
d = b[0]

e = a[:,0,:].unsqueeze(dim=1)
f = b[:,0,:].unsqueeze(dim=2)
g = a[:,1,:].unsqueeze(dim=1)
h = b[:,1,:].unsqueeze(dim=2)

result_1 = torch.matmul(e,f) / torch.sqrt(torch.matmul(e,e.permute(0,2,1)) * torch.matmul(f.permute(0,2,1),f))
result_2 = torch.matmul(g,h) / torch.sqrt(torch.matmul(g,g.permute(0,2,1)) * torch.matmul(h.permute(0,2,1),h))

cos_2 = CosineSimilarity(dim=1)
cos(a,b), cos_2(c,d), result_1.squeeze(), result_2.squeeze()

(tensor([[0.8986, 0.8404],
         [0.8575, 0.3225],
         [0.8550, 0.8219]]),
 tensor([0.8986, 0.8404]),
 tensor([0.8986, 0.8575, 0.8550]),
 tensor([0.8404, 0.3225, 0.8219]))

In [12]:
cos = CosineSimilarity(dim=2)
a = torch.rand((4,3,2,3))
b = torch.rand((4,3,2,3))
a, a[0,:,0,0]

(tensor([[[[0.3279, 0.7263, 0.7824],
           [0.5332, 0.0821, 0.0460]],
 
          [[0.0161, 0.1543, 0.4402],
           [0.9077, 0.7735, 0.8705]],
 
          [[0.0439, 0.1470, 0.4554],
           [0.1357, 0.9111, 0.3551]]],
 
 
         [[[0.4728, 0.3827, 0.7409],
           [0.5877, 0.7028, 0.0544]],
 
          [[0.0658, 0.2674, 0.4417],
           [0.5513, 0.3435, 0.1445]],
 
          [[0.2769, 0.9772, 0.3803],
           [0.9192, 0.6064, 0.4294]]],
 
 
         [[[0.9220, 0.2475, 0.1829],
           [0.7452, 0.8068, 0.5152]],
 
          [[0.8324, 0.4537, 0.0284],
           [0.5481, 0.4037, 0.9052]],
 
          [[0.4446, 0.2342, 0.1037],
           [0.4158, 0.3318, 0.9600]]],
 
 
         [[[0.9149, 0.7405, 0.5799],
           [0.0782, 0.2966, 0.6611]],
 
          [[0.6452, 0.9098, 0.6883],
           [0.8589, 0.1267, 0.3188]],
 
          [[0.3832, 0.5644, 0.1832],
           [0.8251, 0.3839, 0.3712]]]]),
 tensor([0.3279, 0.0161, 0.0439]))

In [11]:
e = gumbel_softmax(c,tau=0.1,hard=True)
f = gumbel_softmax(a,tau=0.1,hard=True)
g = gumbel_softmax(m,tau=0.1,hard=True)
e,f,g

(tensor([[0., 0., 0., 1., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0.]]),
 tensor([[nan, nan, nan, nan, nan, nan]]))

In [2]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import torch
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import GloVe
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,constructBasicDict,run_eval,run_train
from models.NPA import NPAModel

if __name__ == "__main__":    hparams = {
    'mode':"large",
    'batch_size':100,
    'title_size':30,
    'his_size':50,   
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title'],
}

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = '/home/peitian_zhang/Codes/NR/models/model_param/NPA_'+ hparams['mode'] +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)
dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test)

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=20,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)


In [6]:
a = torch.rand((2,1,1,3))
b = torch.rand((2,3,3,1))

a, b, torch.matmul(a,b)

(tensor([[[[0.2782, 0.6213, 0.3146]]],
 
 
         [[[0.5008, 0.1810, 0.9057]]]]),
 tensor([[[[0.6608],
           [0.3568],
           [0.1521]],
 
          [[0.1186],
           [0.2018],
           [0.5153]],
 
          [[0.8943],
           [0.6125],
           [0.4719]]],
 
 
         [[[0.8186],
           [0.5829],
           [0.4187]],
 
          [[0.7143],
           [0.3065],
           [0.9706]],
 
          [[0.5004],
           [0.4140],
           [0.0489]]]]),
 tensor([[[[0.4534]],
 
          [[0.3205]],
 
          [[0.7778]]],
 
 
         [[[0.8947]],
 
          [[1.2923]],
 
          [[0.3698]]]]))